In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1029449,2021-02-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1029450,2021-02-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1029451,2021-02-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1029452,2021-02-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27927,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27929,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27931,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27933,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27935,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
637680,2021-02-24,Arkansas,Arkansas,5001,1963,38.00,5000,Arkansas,AR,Arkansas,State,3017804
637682,2021-02-25,Arkansas,Arkansas,5001,1967,38.00,5000,Arkansas,AR,Arkansas,State,3017804
637684,2021-02-26,Arkansas,Arkansas,5001,1971,38.00,5000,Arkansas,AR,Arkansas,State,3017804
637686,2021-02-27,Arkansas,Arkansas,5001,1980,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1029449,2021-02-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1029450,2021-02-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1029451,2021-02-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1029452,2021-02-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-28') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
404,2021-02-28,Snohomish,Washington,30828,538.00,53061,WA,County,822083,3749.99,65.44
806,2021-02-28,Cook,Illinois,474397,9914.00,17031,IL,County,5150233,9211.18,192.50
1207,2021-02-28,Orange,California,261220,3917.00,6059,CA,County,3175692,8225.61,123.34
1607,2021-02-28,Maricopa,Arizona,510360,9117.00,4013,AZ,County,4485414,11378.21,203.26
2007,2021-02-28,Los Angeles,California,1192559,21435.00,6037,CA,County,10039107,11879.13,213.52
...,...,...,...,...,...,...,...,...,...,...,...
1029021,2021-02-28,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1029160,2021-02-28,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1029268,2021-02-28,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1029372,2021-02-28,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
404,2021-02-28,Snohomish,Washington,30828,538.00,53061,WA,County,822083,3749.99,65.44,65.44,3749.99
806,2021-02-28,Cook,Illinois,474397,9914.00,17031,IL,County,5150233,9211.18,192.50,192.50,9211.18
1207,2021-02-28,Orange,California,261220,3917.00,6059,CA,County,3175692,8225.61,123.34,123.34,8225.61
1607,2021-02-28,Maricopa,Arizona,510360,9117.00,4013,AZ,County,4485414,11378.21,203.26,203.26,11378.21
2007,2021-02-28,Los Angeles,California,1192559,21435.00,6037,CA,County,10039107,11879.13,213.52,213.52,11879.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029021,2021-02-28,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1029160,2021-02-28,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1029268,2021-02-28,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1029372,2021-02-28,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
725387,2021-02-28,Clarke,Alabama,3436,50.00,1025,AL,County,23622,14545.76,211.67,211.67,14545.76,1
666667,2021-02-28,Hale,Alabama,2097,68.00,1065,AL,County,14651,14313.02,464.13,464.13,14313.02,2
522081,2021-02-28,Lowndes,Alabama,1310,51.00,1085,AL,County,9726,13469.05,524.37,524.37,13469.05,3
463094,2021-02-28,Etowah,Alabama,13176,319.00,1055,AL,County,102268,12883.80,311.93,311.93,12883.80,4
388806,2021-02-28,Franklin,Alabama,3991,77.00,1059,AL,County,31362,12725.59,245.52,245.52,12725.59,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636339,2021-02-28,Converse,Wyoming,970,17.00,56009,WY,County,13822,7017.80,122.99,122.99,7017.80,19
636677,2021-02-28,Sublette,Wyoming,676,7.00,56035,WY,County,9831,6876.21,71.20,71.20,6876.21,20
878970,2021-02-28,Niobrara,Wyoming,149,2.00,56027,WY,County,2356,6324.28,84.89,84.89,6324.28,21
790324,2021-02-28,Lincoln,Wyoming,1229,11.00,56023,WY,County,19830,6197.68,55.47,55.47,6197.68,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
522081,2021-02-28,Lowndes,Alabama,1310,51.00,1085,AL,County,9726,13469.05,524.37,524.37,13469.05,3,1
666667,2021-02-28,Hale,Alabama,2097,68.00,1065,AL,County,14651,14313.02,464.13,464.13,14313.02,2,2
461730,2021-02-28,Clay,Alabama,1443,54.00,1027,AL,County,13235,10902.91,408.01,408.01,10902.91,21,3
222581,2021-02-28,Walker,Alabama,6518,255.00,1127,AL,County,63521,10261.17,401.44,401.44,10261.17,32,4
562878,2021-02-28,Greene,Alabama,876,32.00,1063,AL,County,8111,10800.15,394.53,394.53,10800.15,23,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561521,2021-02-28,Hot Springs,Wyoming,360,3.00,56017,WY,County,4413,8157.72,67.98,67.98,8157.72,17,19
770736,2021-02-28,Uinta,Wyoming,2053,12.00,56041,WY,County,20226,10150.30,59.33,59.33,10150.30,4,20
790324,2021-02-28,Lincoln,Wyoming,1229,11.00,56023,WY,County,19830,6197.68,55.47,55.47,6197.68,22,21
221540,2021-02-28,Teton,Wyoming,3351,9.00,56039,WY,County,23464,14281.45,38.36,38.36,14281.45,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027020,2021-02-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1027021,2021-02-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1027022,2021-02-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1027023,2021-02-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
722625,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,211.67,14545.76,46,1,2.00
722626,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,211.67,14545.76,46,1,0.00
722627,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,211.67,14545.76,46,1,1.00
722628,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,211.67,14545.76,46,1,4.00
722629,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,211.67,14545.76,46,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803448,2021-02-24,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
803449,2021-02-25,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
803450,2021-02-26,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
803451,2021-02-27,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
519989,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13469.05,1,3,1.00,0.00
519990,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13469.05,1,3,0.00,0.00
519991,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13469.05,1,3,0.00,0.00
519992,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13469.05,1,3,0.00,0.00
519993,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13469.05,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518284,2021-02-24,Albany,Wyoming,3893,11.00,56001,WY,County,38880,10012.86,28.29,28.29,10043.72,23,6,1.00,0.00
518285,2021-02-25,Albany,Wyoming,3894,11.00,56001,WY,County,38880,10015.43,28.29,28.29,10043.72,23,6,1.00,0.00
518286,2021-02-26,Albany,Wyoming,3901,11.00,56001,WY,County,38880,10033.44,28.29,28.29,10043.72,23,6,7.00,0.00
518287,2021-02-27,Albany,Wyoming,3901,11.00,56001,WY,County,38880,10033.44,28.29,28.29,10043.72,23,6,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-28') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
265577,2021-02-28,Imperial,California,26903,590.00,6025,CA,County,181215,14845.90,325.58,325.58,14845.90,1,2,15.00,0.00
2007,2021-02-28,Los Angeles,California,1192559,21435.00,6037,CA,County,10039107,11879.13,213.52,213.52,11879.13,2,5,1038.00,107.00
464410,2021-02-28,Inyo,California,1275,37.00,6027,CA,County,18039,7068.02,205.11,205.11,7068.02,3,27,4.00,0.00
59357,2021-02-28,Stanislaus,California,56132,946.00,6099,CA,County,550660,10193.59,171.79,171.79,10193.59,4,10,108.00,0.00
73554,2021-02-28,Tulare,California,47869,758.00,6107,CA,County,466195,10268.02,162.59,162.59,10268.02,5,9,0.00,0.00
27567,2021-02-28,Riverside,California,289450,3767.00,6065,CA,County,2470546,11716.03,152.48,152.48,11716.03,6,6,0.00,0.00
47255,2021-02-28,San Joaquin,California,66709,1101.00,6077,CA,County,762148,8752.76,144.46,144.46,8752.76,7,18,0.00,0.00
26849,2021-02-28,Fresno,California,95393,1443.00,6019,CA,County,999101,9547.88,144.43,144.43,9547.88,8,14,191.00,0.00
562820,2021-02-28,Kings,California,22060,220.00,6031,CA,County,152940,14423.96,143.85,143.85,14423.96,9,3,18.00,2.00
420993,2021-02-28,Merced,California,28971,397.00,6047,CA,County,277680,10433.23,142.97,142.97,10433.23,10,8,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
978406,2021-02-28,Lassen,California,5601,19.00,6035,CA,County,30573,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00
265577,2021-02-28,Imperial,California,26903,590.00,6025,CA,County,181215,14845.90,325.58,325.58,14845.90,1,2,15.00,0.00
562820,2021-02-28,Kings,California,22060,220.00,6031,CA,County,152940,14423.96,143.85,143.85,14423.96,9,3,18.00,2.00
132481,2021-02-28,San Bernardino,California,286607,2874.00,6071,CA,County,2180085,13146.60,131.83,131.83,13146.60,12,4,316.00,58.00
2007,2021-02-28,Los Angeles,California,1192559,21435.00,6037,CA,County,10039107,11879.13,213.52,213.52,11879.13,2,5,1038.00,107.00
27567,2021-02-28,Riverside,California,289450,3767.00,6065,CA,County,2470546,11716.03,152.48,152.48,11716.03,6,6,0.00,0.00
166473,2021-02-28,Kern,California,103295,851.00,6029,CA,County,900202,11474.65,94.53,94.53,11474.65,22,7,152.00,24.00
420993,2021-02-28,Merced,California,28971,397.00,6047,CA,County,277680,10433.23,142.97,142.97,10433.23,10,8,0.00,0.00
73554,2021-02-28,Tulare,California,47869,758.00,6107,CA,County,466195,10268.02,162.59,162.59,10268.02,5,9,0.00,0.00
59357,2021-02-28,Stanislaus,California,56132,946.00,6099,CA,County,550660,10193.59,171.79,171.79,10193.59,4,10,108.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
265577,2021-02-28,Imperial,California,26903,590.00,6025,CA,County,181215,14845.90,325.58,325.58,14845.90,1,2,15.00,0.00
2007,2021-02-28,Los Angeles,California,1192559,21435.00,6037,CA,County,10039107,11879.13,213.52,213.52,11879.13,2,5,1038.00,107.00
464410,2021-02-28,Inyo,California,1275,37.00,6027,CA,County,18039,7068.02,205.11,205.11,7068.02,3,27,4.00,0.00
59357,2021-02-28,Stanislaus,California,56132,946.00,6099,CA,County,550660,10193.59,171.79,171.79,10193.59,4,10,108.00,0.00
73554,2021-02-28,Tulare,California,47869,758.00,6107,CA,County,466195,10268.02,162.59,162.59,10268.02,5,9,0.00,0.00
27567,2021-02-28,Riverside,California,289450,3767.00,6065,CA,County,2470546,11716.03,152.48,152.48,11716.03,6,6,0.00,0.00
47255,2021-02-28,San Joaquin,California,66709,1101.00,6077,CA,County,762148,8752.76,144.46,144.46,8752.76,7,18,0.00,0.00
26849,2021-02-28,Fresno,California,95393,1443.00,6019,CA,County,999101,9547.88,144.43,144.43,9547.88,8,14,191.00,0.00
562820,2021-02-28,Kings,California,22060,220.00,6031,CA,County,152940,14423.96,143.85,143.85,14423.96,9,3,18.00,2.00
420993,2021-02-28,Merced,California,28971,397.00,6047,CA,County,277680,10433.23,142.97,142.97,10433.23,10,8,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,325.58,14845.90,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,325.58,14845.90,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,325.58,14845.90,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,325.58,14845.90,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,325.58,14845.90,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3472,02/24/21,Lassen,5594,19.00,18297.19,62.15,62.15,18320.09,38,1,0.00,0.00
3473,02/25/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,7.00,0.00
3474,02/26/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00
3475,02/27/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,325.58,14845.90,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,325.58,14845.90,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,325.58,14845.90,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,325.58,14845.90,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,325.58,14845.90,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3546,02/24/21,Merced,28874,394.00,10398.30,141.89,142.97,10433.23,10,8,125.00,1.00
3547,02/25/21,Merced,28915,395.00,10413.07,142.25,142.97,10433.23,10,8,41.00,1.00
3548,02/26/21,Merced,28971,397.00,10433.23,142.97,142.97,10433.23,10,8,56.00,2.00
3549,02/27/21,Merced,28971,397.00,10433.23,142.97,142.97,10433.23,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)